In [0]:
# !pip install tensorflow-gpu
!pip install onnx
# !pip install torch torchvision
!git clone https://github.com/onnx/onnx-tensorflow.git

In [0]:
import os
os.chdir('/content/onnx-tensorflow/')

In [0]:
!python setup.py install --user

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

'2.1.0'

In [3]:
from onnx_tf.backend import prepare #this module needs 2.x
import onnx
import logging

/root/.local/lib/python3.6/site-packages/onnx_tf-1.5.0-py3.6.egg/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


In [0]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [0]:
from pathlib import Path

In [0]:
onnxPath = Path('/content/model_simple.onnx')
metaPath = Path('/content/model.meta')

In [0]:
def export_tf_meta(onnx_file, meta_file):
    """
    Exports the ONNX model to a Tensorflow Proto file.
    The exported file will have a .meta extension.
    :param onnx_file: string, Path to the .onnx model file
    :param meta_file: string, Path to the exported Tensorflow .meta file
    :return: tuple, input and output tensor dictionaries. Dictionaries have a
        {tensor_name: TF_Tensor_op} structure.
    """
    model = onnx.load(onnx_file)

    # Convert the ONNX model to a Tensorflow graph
    tf_rep = prepare(model)
    output_keys = tf_rep.outputs
    input_keys = tf_rep.inputs

    tf_dict = tf_rep.tensor_dict
    input_tensor_names = {key: tf_dict[key] for key in input_keys}
    output_tensor_names = {key: tf_dict[key] for key in output_keys}

    tf_rep.export_graph(meta_file)
    log.info("Exported Tensorflow proto file to {}".format(meta_file))
    return input_tensor_names, output_tensor_names

In [10]:
!ls /content/

model.meta  model_simple.onnx  onnx-tensorflow	sample_data


In [8]:
input_tensors, output_tensors = export_tf_meta(onnxPath, metaPath)

/root/.local/lib/python3.6/site-packages/onnx_tf-1.5.0-py3.6.egg/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of BitShift in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/root/.local/lib/python3.6/site-packages/onnx_tf-1.5.0-py3.6.egg/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/root/.local/lib/python3.6/site-packages/onnx_tf-1.5.0-py3.6.egg/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ConvInteger in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/root/.local/lib/python3.6/site-packages/onnx_tf-1.5.0-py3.6.egg/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of DequantizeLinear in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/root/.local/lib/python3.6/

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
INFO:__main__:Exported Tensorflow proto file to /content/model.meta


In [0]:
import os

In [9]:
input_tensors ## batchsize, channel, img_size_of_row, img_size_of_col

{'test_input': <tf.Tensor 'test_input:0' shape=(64, 3, 224, 224) dtype=float32>}

In [13]:
output_tensors ## batchsize, labels

{'test_output': <tf.Tensor 'add_18:0' shape=(64, 14) dtype=float32>}